In [10]:
# Issue: 목록 리스트로 돌아간 후, 다른 채용공고 클릭 및 크롤링 안됨. 즉 하나만 크롤링 되고 끝남.

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# ChromeDriver 경로 설정
chrome_driver_path = "C:/Users/chica/OneDrive/바탕 화면/chromedriver-win64/chromedriver.exe"

# WebDriver 옵션 설정
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")

# WebDriver 초기화
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# 페이지 불러오기
driver.get('https://www.work24.go.kr/wk/a/b/1200/retriveDtlEmpSrchList.do?basicSetupYn=&careerTo=&keywordJobCd=&occupation=&seqNo=&cloDateEndtParam=&payGbn=&templateInfo=&rot2WorkYn=&shsyWorkSecd=&srcKeywordParam=&resultCnt=10&keywordJobCont=&cert=&moreButtonYn=&minPay=&codeDepth2Info=11000&currentPageNo=1&eventNo=&mode=&major=&resrDutyExcYn=&eodwYn=&sortField=DATE&staArea=&sortOrderBy=DESC&keyword=&termSearchGbn=&carrEssYns=&benefitSrchAndOr=O&disableEmpHopeGbn=&actServExcYn=&keywordStaAreaNm=&maxPay=&emailApplyYn=&codeDepth1Info=11000&keywordEtcYn=&regDateStdtParam=&publDutyExcYn=&keywordJobCdSeqNo=&viewType=&exJobsCd=&templateDepthNmInfo=&region=&employGbn=&empTpGbcd=&computerPreferential=&infaYn=&cloDateStdtParam=&siteClcd=all&searchMode=Y&birthFromYY=&indArea=&careerTypes=&subEmpHopeYn=&tlmgYn=&academicGbn=&templateDepthNoInfo=&foriegn=&entryRoute=&mealOfferClcd=&basicSetupYnChk=&station=&holidayGbn=&srcKeyword=&academicGbnoEdu=noEdu&enterPriseGbn=all&cloTermSearchGbn=&birthToYY=&keywordWantedTitle=&stationNm=&benefitGbn=&notSrcKeywordParam=&keywordFlag=&notSrcKeyword=&essCertChk=&depth2SelCode=&keywordBusiNm=&preferentialGbn=&rot3WorkYn=&regDateEndtParam=&pfMatterPreferential=&pageIndex=1&termContractMmcnt=&careerFrom=&laborHrShortYn=#scrollLoc')

try:
    with open("../데이터/고용24_채용공고크롤링5.csv", 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['채용공고명', '직무내용', 'URL'])  # CSV 헤더 설정

        page_num = 1  # 현재 페이지 번호

        while page_num <= 3:  # 3페이지까지 크롤링
            item_list_xpath = '//*[@id="contentArea"]//tbody/tr'
            items = driver.find_elements(By.XPATH, item_list_xpath)
            num_items = len(items)  # 해당 페이지의 항목 수

            for i in range(1, num_items + 1):  # 수집한 항목 수만큼 반복
                try:
                    # 새 탭에서 채용 공고가 열리므로 탭을 전환해야 함
                    original_window = driver.current_window_handle  # 현재 창 저장

                    # XPath를 이용해 각 항목 클릭
                    element_xpath = f'//*[@id="list{i}"]/td[2]/a'
                    element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, element_xpath))
                    )
                    driver.execute_script("arguments[0].click();", element)  # JavaScript로 클릭
                    time.sleep(2)  # 페이지 로딩 대기

                    # 새로운 창이 열렸는지 확인하고 그 창으로 이동
                    new_window = [window for window in driver.window_handles if window != original_window][0]
                    driver.switch_to.window(new_window)

                    # 필요한 정보 수집 (XPath 수정된 것 확인)
                    title = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/section/div/div[3]/div[2]/div[1]/div[1]/div[1]/p'))
                    ).text  # 채용공고명
                    desc = driver.find_element(By.XPATH, '//*[@id="contents"]/section/div/div[3]/div[3]/table/tbody/tr').text  # 직무내용
                    job_url = driver.current_url  # 현재 페이지 URL

                    # 수집한 데이터 CSV에 작성
                    writer.writerow([title, desc, job_url])

                    # 새 창 닫고 원래 창으로 돌아가기
                    driver.close()
                    driver.switch_to.window(original_window)
                    time.sleep(2)

                except Exception as e:
                    print(f"Error processing item {i} on page {page_num}: {e}")

            # 페이지 그룹 변경 및 다음 페이지로 이동
            try:
                if page_num % 10 == 0:  # 페이지가 10묶음씩 있음
                    next_button_xpath = '//*[@id="mForm"]/div[2]/div/div[2]/div/div/div/button[13]'  # 다음 버튼 XPath
                else:
                    next_button_xpath = f'//button[@onclick="fn_Move({page_num + 1}); return false;"]'  # 다음 페이지로 이동하는 버튼
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, next_button_xpath))
                )
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(5)  # 페이지 로딩 대기 시간
            except Exception as e:
                print(f"Error navigating to next page: {e}")
                break  # 페이지 이동 중 오류 발생 시 종료

            # 페이지 번호 증가
            page_num += 1

finally:
    driver.quit()


Error processing item 1 on page 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CE2FFDA5+29557]
	(No symbol) [0x00007FF6CE272240]
	(No symbol) [0x00007FF6CE12B6EA]
	(No symbol) [0x00007FF6CE17FA15]
	(No symbol) [0x00007FF6CE17FC6C]
	(No symbol) [0x00007FF6CE1CBB07]
	(No symbol) [0x00007FF6CE1A753F]
	(No symbol) [0x00007FF6CE1C88A3]
	(No symbol) [0x00007FF6CE1A72A3]
	(No symbol) [0x00007FF6CE1712DF]
	(No symbol) [0x00007FF6CE172451]
	GetHandleVerifier [0x00007FF6CE62DCBD+3363469]
	GetHandleVerifier [0x00007FF6CE679B47+3674391]
	GetHandleVerifier [0x00007FF6CE66EAEB+3629243]
	GetHandleVerifier [0x00007FF6CE3BFC66+815670]
	(No symbol) [0x00007FF6CE27D6EF]
	(No symbol) [0x00007FF6CE2792B4]
	(No symbol) [0x00007FF6CE279450]
	(No symbol) [0x00007FF6CE2681FF]
	BaseThreadInitThunk [0x00007FFDFD54257D+29]
	RtlUserThreadStart [0x00007FFDFF30AF08+40]

Error processing item 2 on page 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CE2FFDA5+29557]
	(No symbol) [0x00007FF6CE272240]
	(N

In [9]:
# Issue: 목록 리스트로 돌아간 후, 다른 채용공고 클릭 및 크롤링 안됨. 즉 하나만 크롤링 되고 끝남.

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# ChromeDriver 경로 설정
chrome_driver_path = "C:/Users/chica/OneDrive/바탕 화면/chromedriver-win64/chromedriver.exe"

# WebDriver 옵션 설정
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")

# WebDriver 초기화
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# 페이지 불러오기
driver.get('https://www.work24.go.kr/wk/a/b/1200/retriveDtlEmpSrchList.do?basicSetupYn=&careerTo=&keywordJobCd=&occupation=&seqNo=&cloDateEndtParam=&payGbn=&templateInfo=&rot2WorkYn=&shsyWorkSecd=&srcKeywordParam=&resultCnt=10&keywordJobCont=&cert=&moreButtonYn=&minPay=&codeDepth2Info=11000&currentPageNo=1&eventNo=&mode=&major=&resrDutyExcYn=&eodwYn=&sortField=DATE&staArea=&sortOrderBy=DESC&keyword=&termSearchGbn=&carrEssYns=&benefitSrchAndOr=O&disableEmpHopeGbn=&actServExcYn=&keywordStaAreaNm=&maxPay=&emailApplyYn=&codeDepth1Info=11000&keywordEtcYn=&regDateStdtParam=&publDutyExcYn=&keywordJobCdSeqNo=&viewType=&exJobsCd=&templateDepthNmInfo=&region=&employGbn=&empTpGbcd=&computerPreferential=&infaYn=&cloDateStdtParam=&siteClcd=all&searchMode=Y&birthFromYY=&indArea=&careerTypes=&subEmpHopeYn=&tlmgYn=&academicGbn=&templateDepthNoInfo=&foriegn=&entryRoute=&mealOfferClcd=&basicSetupYnChk=&station=&holidayGbn=&srcKeyword=&academicGbnoEdu=noEdu&enterPriseGbn=all&cloTermSearchGbn=&birthToYY=&keywordWantedTitle=&stationNm=&benefitGbn=&notSrcKeywordParam=&keywordFlag=&notSrcKeyword=&essCertChk=&depth2SelCode=&keywordBusiNm=&preferentialGbn=&rot3WorkYn=&regDateEndtParam=&pfMatterPreferential=&pageIndex=1&termContractMmcnt=&careerFrom=&laborHrShortYn=#scrollLoc')

try:
    with open("../데이터/고용24_채용공고크롤링4.csv", 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['채용공고명', '직무내용', 'URL'])  # CSV 헤더 설정

        page_num = 1  # 현재 페이지 번호

        while page_num <= 3:  # 3페이지까지 크롤링
            item_list_xpath = '//*[@id="contentArea"]/tbody/tr'
            items = driver.find_elements(By.XPATH, item_list_xpath)
            num_items = len(items)  # 해당 페이지의 항목 수

            for i in range(1, num_items + 1):  # 수집한 항목 수만큼 반복
                try:
                    # 새 탭에서 채용 공고가 열리므로 탭을 전환해야 함
                    original_window = driver.current_window_handle  # 현재 창 저장

                    # XPath를 이용해 각 항목 클릭
                    element_xpath = f'//*[@id="list{i}"]/td[2]/a'
                    element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, element_xpath))
                    )
                    driver.execute_script("arguments[0].click();", element)  # JavaScript로 클릭
                    time.sleep(2)  # 페이지 로딩 대기

                    # 새로운 창이 열렸는지 확인하고 그 창으로 이동
                    new_window = [window for window in driver.window_handles if window != original_window][0]
                    driver.switch_to.window(new_window)

                    # 필요한 정보 수집 (XPath 수정된 것 확인)
                    title = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/section/div/div[3]/div[2]/div[1]/div[1]/div[1]/p'))
                    ).text  # 채용공고명
                    desc = driver.find_element(By.XPATH, '//*[@id="contents"]/section/div/div[3]/div[3]/table/tbody/tr').text  # 직무내용
                    job_url = driver.current_url  # 현재 페이지 URL

                    # 수집한 데이터 CSV에 작성
                    writer.writerow([title, desc, job_url])

                    # 새 창 닫고 원래 창으로 돌아가기
                    driver.close()
                    driver.switch_to.window(original_window)
                    time.sleep(2)

                except Exception as e:
                    print(f"Error processing item {i} on page {page_num}: {e}")

            # 페이지 그룹 변경 및 다음 페이지로 이동
            try:
                if page_num % 10 == 0:  # 페이지가 10묶음씩 있음
                    next_button_xpath = '//*[@id="mForm"]/div[2]/div/div[2]/div/div/div/button[13]'  # 다음 버튼 XPath
                else:
                    next_button_xpath = f'//button[@onclick="fn_Move({page_num + 1}); return false;"]'  # 다음 페이지로 이동하는 버튼
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, next_button_xpath))
                )
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(5)  # 페이지 로딩 대기 시간
            except Exception as e:
                print(f"Error navigating to next page: {e}")
                break  # 페이지 이동 중 오류 발생 시 종료

            # 페이지 번호 증가
            page_num += 1

finally:
    driver.quit()


Error processing item 1 on page 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CE2FFDA5+29557]
	(No symbol) [0x00007FF6CE272240]
	(No symbol) [0x00007FF6CE12B6EA]
	(No symbol) [0x00007FF6CE17FA15]
	(No symbol) [0x00007FF6CE17FC6C]
	(No symbol) [0x00007FF6CE1CBB07]
	(No symbol) [0x00007FF6CE1A753F]
	(No symbol) [0x00007FF6CE1C88A3]
	(No symbol) [0x00007FF6CE1A72A3]
	(No symbol) [0x00007FF6CE1712DF]
	(No symbol) [0x00007FF6CE172451]
	GetHandleVerifier [0x00007FF6CE62DCBD+3363469]
	GetHandleVerifier [0x00007FF6CE679B47+3674391]
	GetHandleVerifier [0x00007FF6CE66EAEB+3629243]
	GetHandleVerifier [0x00007FF6CE3BFC66+815670]
	(No symbol) [0x00007FF6CE27D6EF]
	(No symbol) [0x00007FF6CE2792B4]
	(No symbol) [0x00007FF6CE279450]
	(No symbol) [0x00007FF6CE2681FF]
	BaseThreadInitThunk [0x00007FFDFD54257D+29]
	RtlUserThreadStart [0x00007FFDFF30AF08+40]

Error processing item 2 on page 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CE2FFDA5+29557]
	(No symbol) [0x00007FF6CE272240]
	(N

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# ChromeDriver 경로 설정
chrome_driver_path = "C:/Users/chica/OneDrive/바탕 화면/chromedriver-win64/chromedriver.exe"

# WebDriver 옵션 설정
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")

# WebDriver 초기화
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# 페이지 불러오기
driver.get('https://www.work24.go.kr/wk/a/b/1200/retriveDtlEmpSrchList.do?basicSetupYn=&careerTo=&keywordJobCd=&occupation=&seqNo=&cloDateEndtParam=&payGbn=&templateInfo=&rot2WorkYn=&shsyWorkSecd=&srcKeywordParam=&resultCnt=10&keywordJobCont=&cert=&moreButtonYn=&minPay=&codeDepth2Info=11000&currentPageNo=1&eventNo=&mode=&major=&resrDutyExcYn=&eodwYn=&sortField=DATE&staArea=&sortOrderBy=DESC&keyword=&termSearchGbn=&carrEssYns=&benefitSrchAndOr=O&disableEmpHopeGbn=&actServExcYn=&keywordStaAreaNm=&maxPay=&emailApplyYn=&codeDepth1Info=11000&keywordEtcYn=&regDateStdtParam=&publDutyExcYn=&keywordJobCdSeqNo=&viewType=&exJobsCd=&templateDepthNmInfo=&region=&employGbn=&empTpGbcd=&computerPreferential=&infaYn=&cloDateStdtParam=&siteClcd=all&searchMode=Y&birthFromYY=&indArea=&careerTypes=&subEmpHopeYn=&tlmgYn=&academicGbn=&templateDepthNoInfo=&foriegn=&entryRoute=&mealOfferClcd=&basicSetupYnChk=&station=&holidayGbn=&srcKeyword=&academicGbnoEdu=noEdu&enterPriseGbn=all&cloTermSearchGbn=&birthToYY=&keywordWantedTitle=&stationNm=&benefitGbn=&notSrcKeywordParam=&keywordFlag=&notSrcKeyword=&essCertChk=&depth2SelCode=&keywordBusiNm=&preferentialGbn=&rot3WorkYn=&regDateEndtParam=&pfMatterPreferential=&pageIndex=1&termContractMmcnt=&careerFrom=&laborHrShortYn=#scrollLoc')

try:
    with open("../데이터/고용24_채용공고크롤링6.csv", 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(['채용공고명', '직무내용', 'URL'])  # CSV 헤더 설정

        page_num = 1  # 현재 페이지 번호

        while page_num <= 3:  # 3페이지까지 크롤링
            item_list_xpath = '//*[@id="contentArea"]//tbody/tr'
            items = driver.find_elements(By.XPATH, item_list_xpath)
            num_items = len(items)  # 해당 페이지의 항목 수

            for i in range(1, num_items + 1):  # 수집한 항목 수만큼 반복
                try:
                    # 새 탭에서 채용 공고가 열리므로 탭을 전환해야 함
                    original_window = driver.current_window_handle  # 현재 창 저장

                    # XPath를 이용해 각 항목 클릭
                    element_xpath = f'//*[@id="list{i}"]/td[2]/a'
                    element = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, element_xpath))
                    )
                    driver.execute_script("arguments[0].click();", element)  # JavaScript로 클릭
                    time.sleep(2)  # 페이지 로딩 대기

                    # 새로운 창이 열렸는지 확인하고 그 창으로 이동
                    new_window = [window for window in driver.window_handles if window != original_window][0]
                    driver.switch_to.window(new_window)

                    # 필요한 정보 수집 (XPath 수정된 것 확인)
                    title = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/section/div/div[3]/div[2]/div[1]/div[1]/div[1]/p'))
                    ).text  # 채용공고명
                    desc = driver.find_element(By.XPATH, '//*[@id="contents"]/section/div/div[3]/div[3]/table/tbody/tr').text  # 직무내용
                    job_url = driver.current_url  # 현재 페이지 URL

                    # 수집한 데이터 CSV에 작성
                    writer.writerow([title, desc, job_url])

                    # 새 창 닫고 원래 창으로 돌아가기
                    driver.close()
                    driver.switch_to.window(original_window)
                    time.sleep(2)

                except Exception as e:
                    print(f"Error processing item {i} on page {page_num}: {e}")

            # 페이지 그룹 변경 및 다음 페이지로 이동
            try:
                if page_num % 10 == 0:  # 페이지가 10묶음씩 있음
                    next_button_xpath = '//*[@id="mForm"]/div[2]/div/div[2]/div/div/div/button[13]'  # 다음 버튼 XPath
                else:
                    next_button_xpath = f'//button[@onclick="fn_Move({page_num + 1}); return false;"]'  # 다음 페이지로 이동하는 버튼
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, next_button_xpath))
                )
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(5)  # 페이지 로딩 대기 시간
            except Exception as e:
                print(f"Error navigating to next page: {e}")
                break  # 페이지 이동 중 오류 발생 시 종료

            # 페이지 번호 증가
            page_num += 1

finally:
    driver.quit()


Error processing item 1 on page 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CE2FFDA5+29557]
	(No symbol) [0x00007FF6CE272240]
	(No symbol) [0x00007FF6CE12B6EA]
	(No symbol) [0x00007FF6CE17FA15]
	(No symbol) [0x00007FF6CE17FC6C]
	(No symbol) [0x00007FF6CE1CBB07]
	(No symbol) [0x00007FF6CE1A753F]
	(No symbol) [0x00007FF6CE1C88A3]
	(No symbol) [0x00007FF6CE1A72A3]
	(No symbol) [0x00007FF6CE1712DF]
	(No symbol) [0x00007FF6CE172451]
	GetHandleVerifier [0x00007FF6CE62DCBD+3363469]
	GetHandleVerifier [0x00007FF6CE679B47+3674391]
	GetHandleVerifier [0x00007FF6CE66EAEB+3629243]
	GetHandleVerifier [0x00007FF6CE3BFC66+815670]
	(No symbol) [0x00007FF6CE27D6EF]
	(No symbol) [0x00007FF6CE2792B4]
	(No symbol) [0x00007FF6CE279450]
	(No symbol) [0x00007FF6CE2681FF]
	BaseThreadInitThunk [0x00007FFDFD54257D+29]
	RtlUserThreadStart [0x00007FFDFF30AF08+40]

Error processing item 2 on page 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CE2FFDA5+29557]
	(No symbol) [0x00007FF6CE272240]
	(N